# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
from pprint import pprint

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(cities)
len(cities)

['morehead', 'maua', 'lolua', 'qaanaaq', 'kuytun', 'faanui', 'boende', 'nanortalik', 'namibe', 'simpang', 'riverton', 'carutapera', 'road town', 'saint george', 'tomatlan', 'thompson', 'gornopravdinsk', 'calbuco', 'albany', 'shiyan', 'illoqqortoormiut', 'winnemucca', 'rikitea', 'torrington', 'puerto ayora', 'mahe', 'maceio', 'taolanaro', 'strezhevoy', 'vardo', 'carnarvon', 'puerto del rosario', 'casper', 'aquiraz', 'agva', 'tsihombe', 'hellvik', 'west bay', 'tasiilaq', 'east london', 'chernyshevskiy', 'farmington', 'marsaxlokk', 'punta arenas', 'sao filipe', 'san andres', 'port elizabeth', 'saint-francois', 'victoria', 'hermanus', 'nhulunbuy', 'ushuaia', 'belushya guba', 'glencoe', 'narsaq', 'mar del plata', 'busselton', 'samusu', 'hobart', 'kargasok', 'lavrentiya', 'saskylakh', 'angoche', 'grindavik', 'avarua', 'tubuala', 'juneau', 'new norfolk', 'cape town', 'kamenskoye', 'kodiak', 'hilo', 'ribeira grande', 'eldikan', 'hambantota', 'viedma', 'kruisfontein', 'ambulu', 'severo-kurilsk'

614

In [3]:
city_list = cities
city_list

['morehead',
 'maua',
 'lolua',
 'qaanaaq',
 'kuytun',
 'faanui',
 'boende',
 'nanortalik',
 'namibe',
 'simpang',
 'riverton',
 'carutapera',
 'road town',
 'saint george',
 'tomatlan',
 'thompson',
 'gornopravdinsk',
 'calbuco',
 'albany',
 'shiyan',
 'illoqqortoormiut',
 'winnemucca',
 'rikitea',
 'torrington',
 'puerto ayora',
 'mahe',
 'maceio',
 'taolanaro',
 'strezhevoy',
 'vardo',
 'carnarvon',
 'puerto del rosario',
 'casper',
 'aquiraz',
 'agva',
 'tsihombe',
 'hellvik',
 'west bay',
 'tasiilaq',
 'east london',
 'chernyshevskiy',
 'farmington',
 'marsaxlokk',
 'punta arenas',
 'sao filipe',
 'san andres',
 'port elizabeth',
 'saint-francois',
 'victoria',
 'hermanus',
 'nhulunbuy',
 'ushuaia',
 'belushya guba',
 'glencoe',
 'narsaq',
 'mar del plata',
 'busselton',
 'samusu',
 'hobart',
 'kargasok',
 'lavrentiya',
 'saskylakh',
 'angoche',
 'grindavik',
 'avarua',
 'tubuala',
 'juneau',
 'new norfolk',
 'cape town',
 'kamenskoye',
 'kodiak',
 'hilo',
 'ribeira grande',
 'eld

In [6]:
df = pd.DataFrame(city_list)
df["City"]=cities
df["Lat"] = ""
df["Lng"] = ""
df= df.drop([0], axis=1)
df

,City,Lat,Lng
0,morehead,,
1,maua,,
2,lolua,,
3,qaanaaq,,
4,kuytun,,
...,...,...,...
609,carrefour,,
610,naze,,
611,pemangkat,,
612,kiunga,,


In [8]:
df["Temp"]= ""
df["Humidity"] = ""
df["Cloudiness"] = ""
df["WindSpeed"] = ""
df

,City,Lat,Lng,Temp,Humidity,Cloudiness,WindSpeed
0,morehead,,,,,,
1,maua,,,,,,
2,lolua,,,,,,
3,qaanaaq,,,,,,
4,kuytun,,,,,,
...,...,...,...,...,...,...,...
609,carrefour,,,,,,
610,naze,,,,,,
611,pemangkat,,,,,,
612,kiunga,,,,,,


In [9]:
city_short_df = df.head(10)
city_short_df 

,City,Lat,Lng,Temp,Humidity,Cloudiness,WindSpeed
0,morehead,,,,,,
1,maua,,,,,,
2,lolua,,,,,,
3,qaanaaq,,,,,,
4,kuytun,,,,,,
5,faanui,,,,,,
6,boende,,,,,,
7,nanortalik,,,,,,
8,namibe,,,,,,
9,simpang,,,,,,


In [36]:
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"


lat = []
lng = []
temps = []
humidity = []
cloudiness = []
windspeed = []


# looping through test cities:
# for index, row in city_short_df.iterrows():
for index, row in city_short_df.iterrows():
    city = row["City"]
    weather_url = f"{url}appid={weather_api_key}&units={units}&q={city}"
    response = requests.get(weather_url).json()

    
    try:
        
        city_short_df.loc[index, "Lng"] = lng.append(response["coord"]["lon"])
        city_short_df.loc[index, "Temp"] = temps.append(response["main"]["temp_max"])
        city_short_df.loc[index, "Humidity"] = humidity.append(response["main"]["humidity"])
        city_short_df.loc[index, "Cloudiness"] = cloudiness.append(response["clouds"]["all"])
        city_short_df.loc[index, "WindSpeed"] = windspeed.append(response["wind"]["speed"])
        city_short_df.loc[index, "Lat"] = lat.append(response["coord"]["lat"])
        print(f"processing {index}| {city}")
        
    except KeyError:
        print("skipping {city}")


/Users/Tyson/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-36-e7483ddcda23>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_short_df.loc[index, "Lng"] = lng.append(response["coord"]["lon"])
<ipython-input-36-e7483ddcda23>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_short_df.loc[index, "Temp"

processing 0| morehead
processing 1| maua
skipping {city}
processing 3| qaanaaq
processing 4| kuytun
processing 5| faanui
processing 6| boende
processing 7| nanortalik
processing 8| namibe
processing 9| simpang


In [39]:
print(lat)
print(lng)
print(city)
print(temps)
print(humidity)
print(cloudiness)

[37.27, -23.67, 77.48, 54.34, -16.48, -0.22, 60.14, -15.2, -1.27]
[-87.18, -46.46, -69.36, 101.5, -151.75, 20.87, -45.24, 12.15, 104.08]
simpang
[51.8, 57.2, -2.88, 29.26, 79.72, 72.95, 35.8, 67.86, 75.92]
[80, 82, 78, 92, 81, 94, 89, 69, 95]
[1, 40, 74, 100, 100, 99, 100, 0, 100]


In [ ]:
citieslis = {"lat": [lat], "lng": [lng], "Temp": [temps],
             "Humidity": [humidity], "Cloudiness":[cloudiness],
             "windSpeed": [windspeed]}
df["Lat"] = lat
df["Lng"] = lng
df["Temps"] = temps
df["Humidity"] = humidity
df["Cloudiness"] = cloudiness
df["WindSpeed"] = windspeed
df

# temps = cities_df["Max Temp"]
humidity = cities_df["Humidity"]
cloudiness = cities_df["Cloudiness"]
lat = cities_df["Lat"]
windspeed = cities_df["Wind Speed"]

print(north_lat)

In [ ]:
# North Data
n_temps = north_df["Max Temp"]
n_humidity = north_df["Humidity"]
n_cloudiness = north_df["Cloudiness"]
n_windspeed = north_df["Wind Speed"]
n_lat = north_df["Lat"]

# South Data
s_temps = south_df["Max Temp"]
s_humidity = south_df["Humidity"]
s_cloudiness = south_df["Cloudiness"]
s_windspeed = south_df["Wind Speed"]
s_lat = south_df["Lat"]

In [ ]:
plt.scatter(temps, lat)
plt.title("Temp Vs Latitude")
plt.xlabel("Temperature")
plt.ylabel("Latitude")

print("As the latitude reaches the Equator, the tempurature gets warmer")

In [ ]:
plt.scatter(humidity, lat)
plt.title("Humidity Vs Latitude")
plt.xlabel("Humidity")
plt.ylabel("Latitude")

In [ ]:
plt.scatter(cloudiness, lat)
plt.title("Cloudiness Vs Latitude")
plt.xlabel("Cloudiness %")
plt.ylabel("Latitude")

In [ ]:
plt.scatter(windspeed, lat)
plt.title("WindSpeed Vs Latitude")
plt.xlabel("Windspeed (mph)")
plt.ylabel("Latitude")

In [ ]:
# Temperature (F) vs. Latitude
# Northern Hem
plt.subplot(2,2.5,1)
plt.scatter(n_temps, north_lat)
plt.title("Temp Vs Latitude- Northern Hem")


# # Southern Hem
plt.subplot(2,3,3)
plt.scatter(s_temps, south_lat)
plt.title("Temp Vs Latitude- Southern Hem")



In [ ]:
north_df = cities_df.loc[cities_df["Lat"] >0,:]
# north_df
south_df = cities_df.loc[cities_df["Lat"] <0,:]
north_df

In [ ]:
# Humidity vs. Latitude
# Northern Hem
plt.subplot(2,2.5,1)
plt.scatter(n_humidity, north_lat)
plt.title("Temp Vs Latitude- Northern Hem")


# # Southern Hem
plt.subplot(2,3,3)
plt.scatter(s_humidity, south_lat)
plt.title("Temp Vs Latitude- Southern Hem")

In [ ]:
# Cloudiness vs. Latitude
# Northern Hem
plt.subplot(2,2.5,1)
plt.scatter(n_cloudiness, north_lat)
plt.title("Temp Vs Latitude- Northern Hem")


# # Southern Hem
plt.subplot(2,3,3)
plt.scatter(s_cloudiness, south_lat)
plt.title("Temp Vs Latitude- Southern Hem")

In [ ]:
# Wind Speed vs. Latitude
# Northern Hem
plt.subplot(2,2.5,1)
plt.scatter(n_windspeed, north_lat)
plt.title("Temp Vs Latitude- Northern Hem")


# # Southern Hem
plt.subplot(2,3,3)
plt.scatter(s_windspeed, south_lat)
plt.title("Temp Vs Latitude- Southern Hem")

## Generate Cities List

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression